# NeRF Training on Google Colab

**GitHub Repo**: https://github.com/mabuhann/nerf-pytorch

**Dataset**: Google Drive

**Expected Runtime**: 4-6 hours on T4 GPU

## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Clone GitHub Repo

In [ ]:
!git clone https://github.com/mabuhann/nerf-pytorch.git
%cd nerf-pytorch

print("\n✓ Repo cloned")
!ls -lh

## Step 3: Mount Google Drive and Copy Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✓ Drive mounted")

In [ ]:
# Copy dataset from your Google Drive
# Adjust the path if your folder structure is different

import os

# Create data directory
!mkdir -p data/nerf_synthetic

# Copy the lego dataset from Drive
# This assumes you have a 'lego' folder in the Drive link you shared
!cp -r /content/drive/MyDrive/lego data/nerf_synthetic/

# If your Drive structure is different, it might be:
# !cp -r "/content/drive/MyDrive/Your Folder Name/lego" data/nerf_synthetic/

print("\n✓ Dataset copied")

# Verify dataset
print("\nChecking dataset files:")
!ls data/nerf_synthetic/lego/

## Step 4: Install Dependencies

In [ ]:
!pip install imageio imageio-ffmpeg configargparse scikit-image -q

print("✓ Dependencies installed")

## Step 5: Verify Files

In [ ]:
import json

print("Checking files...\n")

# Check Python files
required = ['run_nerf_with_metrics.py', 'run_nerf_helpers.py', 'load_blender.py']
for f in required:
    print(f"{'✓' if os.path.exists(f) else '✗'} {f}")

# Check config
print(f"\n{'✓' if os.path.exists('configs/lego_config.txt') else '✗'} configs/lego_config.txt")

# Check dataset transforms
print("\nDataset:")
for split in ['train', 'val', 'test']:
    path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    if os.path.exists(path):
        with open(path) as f:
            data = json.load(f)
        print(f"✓ {split}: {len(data['frames'])} images")
    else:
        print(f"✗ {split}: not found")

## Step 6: Start Training

**This will take 4-6 hours**

Keep the browser tab open or use Colab Pro to avoid timeouts.

In [ ]:
!python run_nerf_with_metrics.py --config configs/lego_config.txt

## Step 7: View Results

In [ ]:
import json
from IPython.display import Image, display

# Show summary
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    summary = json.load(f)

for key, value in summary.items():
    if value is not None:
        print(f"{key}: {value}")

print("="*60)

# Show plot
print("\nTraining Plot:")
display(Image('logs/lego_metrics/training_metrics.png'))

## Step 8: Analyze Results (Optional)

In [ ]:
# Run detailed analysis
!python analyze_results.py \
    --log_dir logs/lego_metrics \
    --save_plot logs/lego_metrics/detailed_analysis.png

# Show detailed plot
print("\nDetailed Analysis:")
display(Image('logs/lego_metrics/detailed_analysis.png'))

## Step 9: Download Results

In [ ]:
from google.colab import files

# Zip results
!zip -r results.zip logs/lego_metrics/

# Download
files.download('results.zip')

print("\n✓ Download started")

## Step 10: Key Metrics for Report

In [ ]:
# Extract key metrics
with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    metrics = json.load(f)

print("="*60)
print("KEY METRICS FOR YOUR REPORT")
print("="*60)
print(f"Training Time: {metrics['total_training_time_hours']:.2f} hours")
print(f"Test PSNR: {metrics['best_test_psnr']:.2f} dB")
print(f"Test SSIM: {metrics['avg_test_ssim']:.4f}")
print(f"Avg Step Time: {metrics['avg_step_time']:.4f} sec")
print("="*60)